# Building a model to predict the power consumption of a household.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df = pd.read_csv("recs2009_public.csv")


#### DATA ENGINEERING

After reading the file, I ran a few lines of code to simply understand the dataset better, and to see if there was any anomaliess that would immediately catch my eye. 

In [3]:
df.describe()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,PERIODKR,SCALEKER,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
count,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,...,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000,12083.000000
mean,6042.000000,2.628321,5.373086,14.783663,2.659604,9402.981870,4141.375238,1415.406108,4135.146983,1443.503104,...,-1.935943,-1.955392,1681.961764,139.568485,3763.728544,0.499036,5.656046,59.128693,20.256311,90.359431
std,3488.205986,1.042142,2.859366,8.195824,1.192627,5192.101419,2317.759375,1134.659475,2260.543686,1021.748722,...,0.589907,0.415810,1401.079218,264.186908,2597.754419,0.083520,6.797205,8.678293,15.337626,6.166669
min,1.000000,1.000000,1.000000,1.000000,1.000000,476.100000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,-2.000000,0.000000,0.000000,0.000000,0.310000,0.000000,36.000000,-23.000000,60.000000
25%,3021.500000,2.000000,3.000000,8.000000,2.000000,6297.040000,2197.500000,561.000000,2224.000000,712.000000,...,-2.000000,-2.000000,262.500000,4.000000,1246.500000,0.430000,1.000000,52.000000,9.000000,87.000000
50%,6042.000000,3.000000,5.000000,15.000000,2.000000,7970.630000,4483.000000,1045.000000,4502.000000,1179.000000,...,-2.000000,-2.000000,1684.000000,30.000000,3878.000000,0.500000,3.000000,58.000000,18.000000,90.000000
75%,9062.500000,3.000000,7.000000,21.000000,3.000000,11330.030000,5913.000000,1897.000000,5854.000000,1842.500000,...,-2.000000,-2.000000,2662.000000,117.000000,5834.000000,0.560000,7.000000,66.000000,33.000000,94.000000
max,12083.000000,4.000000,10.000000,27.000000,5.000000,95779.140000,12525.000000,5480.000000,13346.000000,5357.000000,...,5.000000,3.000000,7623.000000,1884.000000,11567.000000,0.800000,34.000000,89.000000,67.000000,118.000000


In [4]:
df['KWH'].describe()

count     12083.000000
mean      11288.159398
std        7641.190845
min          17.000000
25%        5837.000000
50%        9623.000000
75%       14765.000000
max      150254.000000
Name: KWH, dtype: float64

I had to convert any categorical data into numeric, so that it could be processed more easily.

In [5]:
layout = pd.read_csv("public_layout.csv")

In [6]:
chars = layout.loc[layout['Variable Type'] == 'Character']
chars

,Variable Name,Variable Label,Variable Order in File,Variable Type,Length
0,DOEID,Unique identifier for each respondent,1,Character,5
12,METROMICRO,Housing unit in Census Metropolitan Statistica...,13,Character,5
13,UR,Housing unit classified as urban or rural by C...,14,Character,1
833,ZTOTSQFT,Imputation flag for TOTSQFT,834,Character,1
834,ZTOTSQFT_EN,Imputation flag for TOTSQFT_EN,835,Character,1
835,ZTOTHSQFT,Imputation flag for TOTHSQFT,836,Character,1
836,ZTOTUSQFT,Imputation flag for TOTUSQFT,837,Character,1
837,ZTOTCSQFT,Imputation flag for TOTCSQFT,838,Character,1
838,ZTOTUCSQFT,Imputation flag for TOTUCSQFT,839,Character,1
931,IECC_Climate_Pub,International Energy Conservation Code (IECC) ...,932,Character,15


In [7]:
uniques = []
for i in chars['Variable Name']:
    uniques.append(df[i].nunique())
uniques

[12083, 3, 2, 2, 2, 2, 2, 2, 2, 11]

In [8]:
catvar = chars['Variable Name']
catvar

0                 DOEID
12           METROMICRO
13                   UR
833            ZTOTSQFT
834         ZTOTSQFT_EN
835           ZTOTHSQFT
836           ZTOTUSQFT
837           ZTOTCSQFT
838          ZTOTUCSQFT
931    IECC_Climate_Pub
Name: Variable Name, dtype: object

I factorized all the categorical values to numerical.

In [9]:
for i in catvar:
    df[i] = pd.factorize(df[i])[0]

In [10]:
df['UR'].nunique()

2

In [11]:
df.shape

(12083, 940)

In [12]:
df.size

11358020

I ran some code to check that there were no irregular NA values and then checked if any data was removed/filtered out.

In [13]:
df = df.dropna() 
# removing invalid entries

In [14]:
df.size

11358020

In [15]:
df.head()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,SCALEKER,IECC_Climate_Pub,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
0,0,2,4,12,2,2471.68,4742,1080,4953,1271,...,-2,0,2117,56,4250,0.48,6,56,9,96
1,1,4,10,26,2,8599.17,2662,199,2688,143,...,-2,1,62,26,2393,0.61,0,64,38,73
2,2,1,1,1,5,8969.92,6233,505,5741,829,...,-2,2,2346,49,5654,0.48,3,52,12,88
3,3,2,3,7,2,18003.64,6034,672,5781,868,...,-2,2,2746,0,4941,0.55,4,55,7,87
4,4,1,1,1,3,5999.61,5388,702,5313,797,...,-2,2,2251,0,5426,0.61,4,50,13,90


In [16]:
list(df.columns)

['DOEID',
 'REGIONC',
 'DIVISION',
 'REPORTABLE_DOMAIN',
 'TYPEHUQ',
 'NWEIGHT',
 'HDD65',
 'CDD65',
 'HDD30YR',
 'CDD30YR',
 'Climate_Region_Pub',
 'AIA_Zone',
 'METROMICRO',
 'UR',
 'KOWNRENT',
 'CONDCOOP',
 'YEARMADE',
 'YEARMADERANGE',
 'OCCUPYYRANGE',
 'CONVERSION',
 'ORIG1FAM',
 'LOOKLIKE',
 'NUMFLRS',
 'NUMAPTS',
 'WALLTYPE',
 'ROOFTYPE',
 'STUDIO',
 'NAPTFLRS',
 'STORIES',
 'TYPEHUQ4',
 'BEDROOMS',
 'NCOMBATH',
 'NHAFBATH',
 'OTHROOMS',
 'TOTROOMS',
 'CELLAR',
 'CRAWL',
 'CONCRETE',
 'BASEFIN',
 'FINBASERMS',
 'BASEHEAT',
 'BASEHT2',
 'PCTBSTHT',
 'BASECOOL',
 'BASECL2',
 'PCTBSTCL',
 'BASEUSE',
 'ATTIC',
 'ATTICFIN',
 'FINATTRMS',
 'ATTCHEAT',
 'ATTCHT2',
 'PCTATTHT',
 'ATTCCOOL',
 'ATTCCL2',
 'PCTATTCL',
 'ATTICUSE',
 'PRKGPLC1',
 'SIZEOFGARAGE',
 'GARGLOC',
 'GARGHEAT',
 'GARGCOOL',
 'PRKGPLC2',
 'SIZEOFDETACH',
 'OUTLET',
 'ZKOWNRENT',
 'ZCONDCOOP',
 'ZYEARMADE',
 'ZYEARMADERANGE',
 'ZOCCUPYYRANGE',
 'ZCONVERSION',
 'ZORIG1FAM',
 'ZLOOKLIKE',
 'ZNUMFLRS',
 'ZNUMAPTS',
 'ZWA

In [17]:
df = df.drop(['DOEID'], axis = 1)
list(df.columns)

['REGIONC',
 'DIVISION',
 'REPORTABLE_DOMAIN',
 'TYPEHUQ',
 'NWEIGHT',
 'HDD65',
 'CDD65',
 'HDD30YR',
 'CDD30YR',
 'Climate_Region_Pub',
 'AIA_Zone',
 'METROMICRO',
 'UR',
 'KOWNRENT',
 'CONDCOOP',
 'YEARMADE',
 'YEARMADERANGE',
 'OCCUPYYRANGE',
 'CONVERSION',
 'ORIG1FAM',
 'LOOKLIKE',
 'NUMFLRS',
 'NUMAPTS',
 'WALLTYPE',
 'ROOFTYPE',
 'STUDIO',
 'NAPTFLRS',
 'STORIES',
 'TYPEHUQ4',
 'BEDROOMS',
 'NCOMBATH',
 'NHAFBATH',
 'OTHROOMS',
 'TOTROOMS',
 'CELLAR',
 'CRAWL',
 'CONCRETE',
 'BASEFIN',
 'FINBASERMS',
 'BASEHEAT',
 'BASEHT2',
 'PCTBSTHT',
 'BASECOOL',
 'BASECL2',
 'PCTBSTCL',
 'BASEUSE',
 'ATTIC',
 'ATTICFIN',
 'FINATTRMS',
 'ATTCHEAT',
 'ATTCHT2',
 'PCTATTHT',
 'ATTCCOOL',
 'ATTCCL2',
 'PCTATTCL',
 'ATTICUSE',
 'PRKGPLC1',
 'SIZEOFGARAGE',
 'GARGLOC',
 'GARGHEAT',
 'GARGCOOL',
 'PRKGPLC2',
 'SIZEOFDETACH',
 'OUTLET',
 'ZKOWNRENT',
 'ZCONDCOOP',
 'ZYEARMADE',
 'ZYEARMADERANGE',
 'ZOCCUPYYRANGE',
 'ZCONVERSION',
 'ZORIG1FAM',
 'ZLOOKLIKE',
 'ZNUMFLRS',
 'ZNUMAPTS',
 'ZWALLTYPE',
 

In [18]:
# for i in df.dtypes:
#     if df.dtypes[1] == 'char':
#         print('flag')

#### PROBLEM METHODOLOGY

After taking a look at the data, visually, I chose to run the pandas function 'corr' to find the correlation of each column, to see which ones were affecting my target feature the most. I used to correlation methods, the pearson and spearman methods, in order to see which one would also yield better features to build my model with.
I highlighted only the columns which had a correlation of 0.5 or higher in order to make my selection.

In [19]:
cor= df.corr()

cor2= df.corr(method='spearman')


In [20]:
# cor['KWH']['ZTOTHSQFT']

In [21]:
# plt.figure(figsize=(12,10))
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()

In [41]:
cor_target = abs(cor["KWH"])
relevant_features = cor_target[cor_target>0.5]
relevant_features

KWH            1.000000
KWHCOL         0.655104
KWHWTH         0.585656
KWHRFG         0.638621
KWHOTH         0.880403
BTUEL          1.000000
BTUELCOL       0.655104
BTUELWTH       0.585656
BTUELRFG       0.638621
BTUELOTH       0.880403
DOLLAREL       0.868712
DOLELCOL       0.601626
DOLELWTH       0.514613
DOLELOTH       0.702251
TOTALBTUCOL    0.655103
TOTALBTURFG    0.638621
TOTALBTUOTH    0.676108
TOTALDOL       0.627922
TOTALDOLCOL    0.601628
TOTALDOLOTH    0.655742
Name: KWH, dtype: float64

In [23]:
cor_target = abs(cor2["KWH"])
relevant_featuresSpear = cor_target[cor_target>0.5]
relevant_featuresSpear

TOTCSQFT       0.509591
KWH            1.000000
KWHCOL         0.622991
KWHRFG         0.629723
KWHOTH         0.871444
BTUEL          1.000000
BTUELCOL       0.622991
BTUELRFG       0.629724
BTUELOTH       0.871444
DOLLAREL       0.903403
DOLELCOL       0.582043
DOLELOTH       0.745624
TOTALBTUCOL    0.622992
TOTALBTURFG    0.629723
TOTALBTUOTH    0.710120
TOTALDOL       0.618778
TOTALDOLCOL    0.582085
TOTALDOLWTH    0.517203
TOTALDOLOTH    0.690908
Name: KWH, dtype: float64

In [24]:
type(relevant_features)
feats = relevant_features.index
feats

Index(['KWH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'BTUEL', 'BTUELCOL',
       'BTUELWTH', 'BTUELRFG', 'BTUELOTH', 'DOLLAREL', 'DOLELCOL', 'DOLELWTH',
       'DOLELOTH', 'TOTALBTUCOL', 'TOTALBTURFG', 'TOTALBTUOTH', 'TOTALDOL',
       'TOTALDOLCOL', 'TOTALDOLOTH'],
      dtype='object')

In [25]:
featsSp = relevant_featuresSpear.index
featsSp

Index(['TOTCSQFT', 'KWH', 'KWHCOL', 'KWHRFG', 'KWHOTH', 'BTUEL', 'BTUELCOL',
       'BTUELRFG', 'BTUELOTH', 'DOLLAREL', 'DOLELCOL', 'DOLELOTH',
       'TOTALBTUCOL', 'TOTALBTURFG', 'TOTALBTUOTH', 'TOTALDOL', 'TOTALDOLCOL',
       'TOTALDOLWTH', 'TOTALDOLOTH'],
      dtype='object')

The features selected were all the highly correlated features, not including BTUEL since it is derived directly from KWH.
I then trained four different models. One set using the features of the pearson correlation, and the other using the features from the spearman correlation. For each of those sets I tried a random forest model, and a decision tree model.

Each model was validated using MAE scores (Mean Absolute Error).

#### The first set was the pearson correlation set.

In [26]:
y = df.KWH
features = ['KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'BTUELCOL',
       'BTUELWTH', 'BTUELRFG', 'BTUELOTH', 'DOLLAREL', 'DOLELCOL', 'DOLELWTH',
       'DOLELOTH', 'TOTALBTUCOL', 'TOTALBTURFG', 'TOTALBTUOTH', 'TOTALDOL',
       'TOTALDOLCOL', 'TOTALDOLOTH']
X = df[features]

In [27]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [28]:
kwh_model = DecisionTreeRegressor(random_state=1)

kwh_model.fit(train_X, train_y)

val_predictions = kwh_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE (when not specifying max_leaf_nodes) for DescisionTree: {:,.0f}".format(val_mae))

Validation MAE (when not specifying max_leaf_nodes) for DescisionTree: 1,038


In [29]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X,train_y)
predict = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(predict,val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 638.8420887123468


#### The second set was the spearman correlation set.

In [30]:
y1 = df.KWH
features2 = ['TOTCSQFT', 'KWHCOL', 'KWHRFG', 'KWHOTH', 'BTUELCOL',
       'BTUELRFG', 'BTUELOTH', 'DOLLAREL', 'DOLELCOL', 'DOLELOTH',
       'TOTALBTUCOL', 'TOTALBTURFG', 'TOTALBTUOTH', 'TOTALDOL', 'TOTALDOLCOL',
       'TOTALDOLWTH', 'TOTALDOLOTH']
X1 = df[features2]

In [31]:
train_X, val_X, train_y, val_y = train_test_split(X1, y1, random_state=1)

In [32]:
kwh_model = DecisionTreeRegressor(random_state=1)

kwh_model.fit(train_X, train_y)

val_predictions = kwh_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for Descision Tree: {:,.0f}".format(val_mae))

Validation MAE for Descision Tree: 994


In [33]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X,train_y)
predict = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(predict,val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 572.8250380668652


The Random forest model using the spearman set of features performed the best, with a MAE score of 572.83. 

The mean KWH being 11288.16 and the error being 572.83, the error margin is about 5.07% 

##### Next, I used XGboost gradient boosting for my model.

In [34]:
from xgboost import XGBRegressor

In [35]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, 
             early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

#### PROBLEM SOLUTION 

In [36]:
predictions = my_model.predict(val_X)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, val_y)))

Mean Absolute Error: 389.332809079526


Ofcourse, the gradient boosted model performs even better, giving a MAE score of 389.33 or 3.45% error margin of the mean.

In [37]:
 from sklearn.metrics import r2_score

In [38]:
r2 = r2_score(val_y, predictions)
r2

0.9854225611786027

Lastly, the R-squared score was also checked, to verify the performance of the model, which gave us a high value of almost 0.99 !

Of course, this could be due to being overfit, and possibly would only return such a high success rate for in-data values.